# Longitiude and Latitude of Toronto Neighborhoods

First we import the necessary libraries

In [2]:
import requests
!conda install -c anaconda lxml --yes
import lxml.html as lh
import pandas as pd

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    blas-1.0                   |              mkl           6 KB  anaconda
  

Next, we access the URL with the dataset

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Then we assign a variable to the the web contents
page = requests.get(url)
#We can then store those contents in a doc variable
doc = lh.fromstring(page.content)
#We then get the contnets within the tr data field
tr_elements = doc.xpath('//tr')

Then we check row width to make sure the table dimensions look accurate

In [4]:
#Check row length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

All are length 3, which is what we would expect, meaning we have grabbed the right table. <br>
Then we take the elements in the first row and make them our headers.

In [5]:
tr_elements = doc.xpath('//tr')
#Create list
col=[]
i=0
#store element and list for each row
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name, []))
    
    


1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Next we put the data into a pandas dataframe

In [6]:
#We store the data starting in the second row (after the header)
for j in range(1,len(tr_elements)):
    #T is equal to row j
    T=tr_elements[j]
    
    #If row is greater than 3 columns then don't include as it isnt from the table we need
    if len(T)!=3:
        break
    

        
    #create index i
    i=0
    
    #Run through each row
    for t in T.iterchildren():
        data=t.text_content() 
        #See if there is an empty row
        if i>0:
        #Convert any data that happens to show up numerical as integer
            try:
                data=str(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

We double check column length to ensure accuracy

In [7]:
[len(C) for (title,C) in col]

[287, 287, 287]

Next we create the dataframe

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

We check to make sure the data imported correctly

In [9]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


The data came in with a newline character in the last column. We need to remove it.

In [10]:
df = df.replace(r'\n',' ', regex=True) 
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now we need to rename the head without the newline character.

In [11]:
df = df.rename(columns={'Neighbourhood\n': 'Neighbourhood'})
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Next, we remove the 'not assigned' boroughs.

In [12]:
# Get names of indexes where borough has a value of "not assigned"
indexes = df[ df['Borough'] == 'Not assigned' ].index
# Delete these row indexes from dataFrame
df.drop(indexes , inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Next we group the neighborhoods of the same postal code. We'll also rename the data frame here in case we need to go back to a point of reference.

In [13]:
df_Toronto =df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Next, we assign the borough name as the neighborhood name when neighborhood is not assigned. 

In [14]:
df_Toronto.loc[df_Toronto['Neighbourhood']=='Not assigned','Neighbourhood'] = df_Toronto.loc[df_Toronto['Neighbourhood']=='Not assigned']['Borough']
df_Toronto.head(10)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


Next we import the numpy library to use the shape function to determine the total number of rows and columns in the final table.

In [15]:
import numpy as np

We call the shape function:

In [16]:
print(df_Toronto.shape)

(103, 3)


The table has 103 rows and 3 columns.

Next we use the Longitude Latitude CSV file to get the latitude and longitude of each postal code. We do not use geocoder to automatically get postcodes because it did not function properly upon use.

In [45]:
df_longlat = pd.read_csv (r'http://cocl.us/Geospatial_data')
#rename column so we can do a join
df_ll=df_longlat.rename(columns={'Postal Code': 'Postcode'})

print (df_ll)


    Postcode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476
..       ...        ...        ...
98       M9N  43.706876 -79.518188
99       M9P  43.696319 -79.532242
100      M9R  43.688905 -79.554724
101      M9V  43.739416 -79.588437
102      M9W  43.706748 -79.594054

[103 rows x 3 columns]


Next we merge the two dataframes on postcode, the common column.

In [48]:


df_combined = pd.merge(df_Toronto,
                 df_ll[['Postcode', 'Latitude', 'Longitude']],
                 on='Postcode')
df_combined.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
